In [1]:
import numpy as np
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test


In [307]:
predictions = {}

In [308]:
stf_pred = {}
stf_pred[1] = np.load("../code/baseline/result/stf-pred-12.npy").item()[1]
stf_pred[3] = np.load("../code/baseline/result/stf-pred-34.npy").item()[3]
predictions['stf'] = stf_pred

In [309]:
mtf_pred = {}
mtf_pred[1] = np.load("../code/baseline/result/mtf-pred-12.npy").item()[1]
mtf_pred[3] = np.load("../code/baseline/result/mtf-pred-34.npy").item()[3]
predictions['mtf'] = mtf_pred

In [310]:
active_threshold = {'hvac':230, 'fridge':20, 'dr':250, 'dw':55, 'mw':10}

In [311]:
num_homes = {0:14, 1:14, 2:14, 3:13, 4:13}

In [312]:
# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]

In [313]:
active = {}
for dataset in [1, 3]:
    active[dataset] = {}
    for fold_num in range(5):
        active[dataset][fold_num] = {}
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active[dataset][fold_num][appliance] = (test_gt[fold_num][appliance] > active_threshold[appliance])*1

In [314]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (stf_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [315]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.811093983185 Recall:  0.977586308995
fridge Precision:  0.978629785109 Recall:  0.920857835046
dr Precision:  0.125 Recall:  0.05203969129
dw Precision:  0.0887822435513 Recall:  0.0397689103856
mw Precision:  0.174036595509 Recall:  0.484632086852
hvac Precision:  0.796154888292 Recall:  0.956993545331
fridge Precision:  0.977819653745 Recall:  0.96098003122
dr Precision:  0.113482056256 Recall:  0.0386990077178
dw Precision:  0.0626020685901 Recall:  0.0463522773075
mw Precision:  0.15893247776 Recall:  0.262919179735


In [316]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (mtf_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [317]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.816622527261 Recall:  0.972323384852
fridge Precision:  0.978581370906 Recall:  0.96372161205
dr Precision:  0.125177003682 Recall:  0.0487320837927
dw Precision:  0.0813429472735 Recall:  0.0485019481392
mw Precision:  0.20485015241 Recall:  0.366417370326
hvac Precision:  0.795939687083 Recall:  0.975676555931
fridge Precision:  0.978608407768 Recall:  0.985699019185
dr Precision:  0.106988783434 Recall:  0.0410143329658
dw Precision:  0.0734101579172 Recall:  0.0693268843208
mw Precision:  0.212929915639 Recall:  0.379975874548


In [318]:
sc_pred = np.load("../code/baseline/result/sc-non-disc-pred.npy").item()
predictions['sc'] = sc_pred

In [319]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (sc_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [320]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  1.0 Recall:  0.924735003257
fridge Precision:  0.983653614 Recall:  0.615462515876
dr Precision:  0.179126572909 Recall:  0.0266813671444
dw Precision:  0.0703951506834 Recall:  0.16539029961
mw Precision:  0.1330680745 Recall:  0.414716525935
hvac Precision:  1.0 Recall:  0.955786995914
fridge Precision:  0.978929010105 Recall:  0.807110205954
dr Precision:  0.136469221835 Recall:  0.15545755237
dw Precision:  0.0717382110383 Recall:  0.364637914819
mw Precision:  0.126826346266 Recall:  0.704897466828


In [321]:
dsc_pred = np.load("../code/baseline/result/DSC-pred.npy").item()
predictions['dsc'] = dsc_pred

In [322]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (dsc_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [323]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  1.0 Recall:  0.979414638479
fridge Precision:  0.984273170267 Recall:  0.462224932719
dr Precision:  0.14131501472 Recall:  0.0158765159868
dw Precision:  0.0785828449654 Recall:  0.0396345559586
mw Precision:  0.176894833464 Recall:  0.195271411339
hvac Precision:  1.0 Recall:  0.981139337952
fridge Precision:  0.98356771308 Recall:  0.392499258654
dr Precision:  0.12187088274 Recall:  0.0407938257993
dw Precision:  0.0633285797285 Recall:  0.161090957947
mw Precision:  0.171051089759 Recall:  0.281737032569


In [324]:
cnn_ind_params = {}
cnn_ind_params[1] = np.load("../code/baseline/result/cnn-individual-new-param-1.npy").item()
cnn_ind_params[3] = np.load("../code/baseline/result/cnn-individual-new-param-3.npy").item()


In [347]:
cnn_ind_pred = {}
for dataset in [1, 3]:
    cnn_ind_pred[dataset] = {}
    for fold_num in range(5):
        cnn_ind_pred[dataset][fold_num] = {}
        iters = cnn_ind_params[dataset][fold_num]['iters']
        lr = cnn_ind_params[dataset][fold_num]['lr']
        for appliance in APPLIANCE_ORDER[1:-1]:
            directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num, lr)
            filename = "test-pred-[\'{}\'].npy".format(appliance)
            try:
                cnn_ind_pred[dataset][fold_num][appliance] = np.load(directory+filename).item()[iters][0].reshape(-1, 112, 24)
            except:
                print("CUDA_VISIBLE_DEVICES={} python cnn-tree-save-test.py {} {} 200000 0 {} {} &".format(fold_num, dataset, lr, fold_num, appliance))



In [348]:
predictions['ind cnn'] = cnn_ind_pred

In [349]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (cnn_ind_pred[dataset][fold_num][appliance] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [350]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(recall)


0.972634274886
0.996044290518
0.457221609702
0.000537417707913
0.00617611580217
0.956016462368
0.996743673383
0.0901874310915
0.00201531640468
0.00598311218335


In [351]:
cnn_tree_params = {}
cnn_tree_params[1] = np.load("../code/baseline/result/cnn-tree-new-param-1.npy").item()
cnn_tree_params[3] = np.load("../code/baseline/result/cnn-tree-final-param-3.npy").item()

In [352]:
# get prediction for cnn tree
cnn_tree_pred = {}
for dataset in [1, 3]:
    cnn_tree_pred[dataset] = {}
    for fold_num in range(5):
        cnn_tree_pred[dataset][fold_num] = {}

        lr = cnn_tree_params[dataset][fold_num]['lr']
        iters = cnn_tree_params[dataset][fold_num]['iters']
        order = cnn_tree_params[dataset][fold_num]['order']

        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
        filename = "test-pred-[\'{}\'].npy".format(o)

        full_path = directory + filename
        cnn_pred = np.asarray(np.load(full_path).item()[iters])

        for idx, appliance in enumerate(order):
            cnn_tree_pred[dataset][fold_num][appliance] = cnn_pred[idx].reshape(-1, 112,24)
    

In [361]:
predictions['tree cnn'] = cnn_tree_pred

In [353]:
for dataset in [1,3]:
    for idx, appliance in enumerate(order):
        error = 0
        for fold_num in range(5):
        
            abs_error = np.abs(cnn_tree_pred[dataset][fold_num][appliance] - test_gt[fold_num][appliance])
            error += abs_error.mean()*num_homes[fold_num]
        print(error/68)

87.5339624734
163.315904141
8.63536474212
16.4218572378
37.7565564686
69.0058276011
306.384807266
9.00085921228
15.4479628075
33.7374402039


In [354]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (cnn_tree_pred[dataset][fold_num][appliance] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [355]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(recall)


0.969777047433
0.969730709622
0.492613009923
0.000537417707913
0.0135102533172
0.959036536981
0.991607405625
0.148732083793
0.0
0.0248974668275


/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [356]:
num_folds=5
def mean_predictions(dataset):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
#         print(pred.shape)
        
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
#         print(pred[0, 1, 0, 20])
        out.append(pred)
#     return np.concatenate(out)
    return out


In [362]:
tensor = np.load('../2015-5appliances-true-agg.npy')
# tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

dataset = 1
mean_pred = {}
for dataset in [1, 3]:

    gt = tensor[:, 1:, :, :]
    mean_pred[dataset] = mean_predictions(dataset)
# Clamping prediction to aggregate
predictions['mean'] = mean_pred

In [358]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (mean_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [359]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.753081105573 Recall:  0.984218629715
fridge Precision:  0.977815344888 Recall:  1.0
dr Precision:  0.10296474359 Recall:  0.0283351708931
dw Precision:  0.0756002595717 Recall:  0.031304581486
mw Precision:  0.170584519218 Recall:  0.495102533172
hvac Precision:  0.753081105573 Recall:  0.984218629715
fridge Precision:  0.977815344888 Recall:  1.0
dr Precision:  0.10296474359 Recall:  0.0283351708931
dw Precision:  0.0756002595717 Recall:  0.031304581486
mw Precision:  0.170584519218 Recall:  0.495102533172


In [360]:
# error of active parts

In [399]:
error = {}
for dataset in [1, 3]:
    error[dataset] = {}
    for method in ['stf', 'mtf', 'sc', 'dsc', 'mean']:
        error[dataset][method] = {}
        print("Dataset ", dataset)
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active_error = []
            inactive_error = []
            print(appliance)
            for fold_num in range(5):
                abs_error = np.abs(predictions[method][dataset][fold_num][:, idx]-test_gt[fold_num][appliance])
                active_error.append(abs_error[active[dataset][fold_num][appliance] == 1])
                inactive_error.append(abs_error[active[dataset][fold_num][appliance] == 0])
    #             print(appliance, fold_num, abs_error[active[dataset][fold_num][appliance] == 1].mean())
            print("active error", np.hstack(active_error).mean())
            print("inactive error", np.hstack(inactive_error).mean())


Dataset  1
hvac
active error 312.920953713
inactive error 352.311600154
fridge
active error 43.4001747279
inactive error 46.7066317886
dr
active error 1198.99797076
inactive error 60.3982116189
dw
active error 348.461856826
inactive error 14.4282359396
mw
active error 48.1898233036
inactive error 6.68728489695
Dataset  1
hvac
active error 318.125981413
inactive error 347.42875228
fridge
active error 40.2023032729
inactive error 55.4443076088
dr
active error 1198.64924853
inactive error 61.8406305985
dw
active error 347.193082631
inactive error 15.0063319151
mw
active error 50.3314639219
inactive error 4.93792805665
Dataset  1
hvac
active error 276.623251431
inactive error 4.35813867811
fridge
active error 118.957349014
inactive error 79.8034176483
dr
active error 1259.90720008
inactive error 19.2902113243
dw
active error 341.543323579
inactive error 17.7608014526
mw
active error 62.6150733512
inactive error 21.8516790999
Dataset  1
hvac
active error 233.605222874
inactive error 0.74415

In [173]:
active_error

[array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]